<a href="https://colab.research.google.com/github/HilbertN/Analisis_de_datos_Tareas/blob/main/RN10_Regresion_GradientTape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

In [15]:
# Blood Pressure data
x = [22, 41, 52, 23, 41, 54, 24, 46, 56, 27, 47, 57, 28, 48, 58,  9,
     49, 59, 30, 49, 63, 32, 50, 67, 33, 51, 71, 35, 51, 77, 40, 51, 81]
y = [131, 139, 128, 128, 171, 105, 116, 137, 145, 106, 111, 141, 114,
     115, 153, 123, 133, 157, 117, 128, 155, 122, 183,
     176,  99, 130, 172, 121, 133, 178, 147, 144, 217]
x = np.asarray(x, np.float32)
y = np.asarray(y, np.float32)

In [18]:
def loss(a, b):
  y_hat = a*x + b
  return tf.reduce_mean((y_hat - y)**2)

Aqui se muestra como tensorflow V2 usa "eager execution". Es decir, asigna valores a los nodos del la Grafica que representa la funcion tan pronto como puede.

In [19]:
a = tf.Variable(0.0)
b = tf.Variable(139.0)
loss(a,b)

<tf.Tensor: shape=(), dtype=float32, numpy=673.4545>

In [20]:
a = tf.Variable(0.0)
b = tf.Variable(139.0)
with tf.GradientTape() as tape:
  loss_val = loss(a,b)
  print("Loss at ", loss_val)
grad_a, grad_b = tape.gradient(loss_val, [a,b])
print(grad_a, grad_b)

Loss at  tf.Tensor(673.4545, shape=(), dtype=float32)
tf.Tensor(-553.09094, shape=(), dtype=float32) tf.Tensor(0.727273, shape=(), dtype=float32)


In [10]:
a = tf.Variable(0.)
b = tf.Variable(139.0)
eta = 0.0004
start= time.time()
for i in range(8000):
  with tf.GradientTape() as tape:
    y_hat = a*x + b
    loss = tf.reduce_mean((y_hat - y)**2)
  grad_a, grad_b  = tape.gradient(loss, [a,b])
  a = tf.Variable(a - eta * grad_a)
  b = tf.Variable(b - eta * grad_b)
  if (i % 5000 == 0):
      t = time.time() - start
      print("Epoch:",i, "slope=",a.numpy(),"intercept=",b.numpy(),"gradient_a", grad_a.numpy(), "gradient_b",grad_b.numpy(), "mse=", loss.numpy(), "time for 1000 epochs ", t/5.)
      start = time.time()

Epoch: 0 slope= 0.22123638 intercept= 138.99971 gradient_a -553.09094 gradient_b 0.727273 mse= 673.4545 time for 1000 epochs  0.003197336196899414
Epoch: 5000 slope= 0.47009143 intercept= 120.60784 gradient_a -0.14053345 gradient_b 7.3059855 mse= 469.57272 time for 1000 epochs  5.278709411621094


Para mejorar el tiempo de cómputo, se puede usar "Lazy Evaluation" que es lo contrario de "Eager Evaluation". Es decir, no asigna valores en la Grafica que representa la función y por lo tanto la "misma gráfica" se puede usar en cada evaluación particular. Esto, elimina el costo computacional de crear una nueva gráfica cada vez que se evalúa la funcion.

Para esta "configuración" simplemente es neceario usar el decorador @tf.function.

In [14]:
start= time.time()
a  = tf.Variable(0.0)
b = tf.Variable(139.0)
eta = 0.0004

@tf.function #Will tell tf to build a graph from this code
def train_step():
    y_hat = a*x + b
    loss = tf.reduce_mean((y_hat - y)**2)
    grad_a, grad_b  = tape.gradient(loss, [a,b])
    a.assign(a - eta * grad_a)
    b.assign(b - eta * grad_b)

for i in range(8000):
  with tf.GradientTape() as tape: #Record the gradients from now on
    train_step()
    if (i % 5000 == 0):
        t = time.time() - start
        print("Epoch:",i, "slope=",a.numpy(),"intercept=",b.numpy(),"gradient_a", "mse=", loss.numpy(), "time for 1000 epochs ", t/5.)
        start = time.time()


Epoch: 0 slope= 0.22123638 intercept= 138.99971 gradient_a mse= 419.89865 time for 1000 epochs  0.014073324203491212
Epoch: 5000 slope= 0.47009143 intercept= 120.60784 gradient_a mse= 419.89865 time for 1000 epochs  0.4604238510131836
